In [203]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

In [204]:
#split dataset
path = "/Users/giovanna/Desktop/Magistrale/SecondoAnno/AIBioInfo/Labs/LAB2_BNN/dataset_LUMINAL_A_B.csv"
test_df = pd.read_csv(path)

In [205]:
print(test_df.shape)

(100, 1023)


In [206]:
test_df['l'] = test_df['l'].str.replace(r'\s+', '', regex=True)
label_mapping = {
    'LuminalB': 1.0,
    'LuminalA': 0.0,
}
# uncomment if you want to use full dataset
#test_df['l'] = test_df['l'].map(label_mapping).values
test_df['l'] = test_df['l'].map(label_mapping)
test_df = test_df.sample(frac=1).reset_index(drop=True)

Normalize values of data

In [207]:
scaler = StandardScaler()
test_df.iloc[:, 1:] = scaler.fit_transform(test_df.iloc[:, 1:])

/var/folders/hx/s2fpl8513bj8s4tk49n3h1lc0000gn/T/ipykernel_23104/66482838.py:2: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  test_df.iloc[:, 1:] = scaler.fit_transform(test_df.iloc[:, 1:])


In [208]:
test_df = test_df.to_numpy()

In [209]:
# now split dataset into train and test
train_p = int(test_df.shape[0] / 100 * 80)
train_data = test_df[:train_p]
test_data = test_df[train_p:]

In [210]:
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.features = torch.tensor(data[:, 1:], dtype=torch.float32)
        self.labels = torch.tensor(data[:, 0], dtype=torch.long)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [211]:
train_dataset = MyDataset(train_data)
test_dataset = MyDataset(test_data)
batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Now I will create my dummy network to perform bayesian classification

In [220]:
import torch
import torch.nn as nn
import torch.optim as optim


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = train_data.shape[1] - 1
model = nn.Sequential(
    nn.Linear(num_features, 128),   # First fully connected layer
    nn.ReLU(),             # ReLU activation
    nn.Linear(128, 64),    # Second fully connected layer
    nn.ReLU(),             # ReLU activation
    nn.Linear(64, 2),      # Output layer for binary classification (2 classes)
)

# Define the loss function for binary classification
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [221]:
num_epochs = 5
for epoch in range(num_epochs):
    for batch in train_loader:  # Iterate over batches
        optimizer.zero_grad()  # Zero the gradients
        batch_images, batch_labels = batch  # Unpack the batch
        outputs = model(batch_images)  # Forward pass
        loss = criterion(outputs, batch_labels)  # Compute the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}')

# Inference
model.eval()  # Set the model to evaluation mode

correct = 0
total = 0

with torch.no_grad():  # Disable gradient tracking for inference
    for batch_images, batch_labels in test_loader:  # Iterate over batches
        outputs = model(batch_images)
        _, predicted = torch.max(outputs.data, 1)
        total += batch_labels.size(0)
        correct += (predicted == batch_labels).sum().item()

print(f'Accuracy on the test dataset: {100 * correct / total}%')

Epoch [1/5], Loss: 0.6671648621559143
Epoch [2/5], Loss: 0.41883522272109985
Epoch [3/5], Loss: 0.30020594596862793
Epoch [4/5], Loss: 0.16912585496902466
Epoch [5/5], Loss: 0.02293531410396099
Accuracy on the test dataset: 70.0%
